![logo](https://imaging.epfl.ch/resources/logo-for-gitlab.svg)

<h1>Imaging Lunch: Introduction to Python for Image Analysis</h1>

This notebook introduces the basics of image handling and manipulation in Python, using <a href="https://scikit-image.org/">Scikit-image</a>, <a href="https://matplotlib.org/stable/">Matplotlib</a>, and <a href="https://numpy.org/">Numpy</a>.

➡️ <a href="https://github.com/EPFL-Center-for-Imaging/ImagingLunch_ImagesInPython">Git repository</a>.

```{admonition} How to use this notebook?
This document is an interactive <a href="https://jupyter.org/">Jupyter notebook</a>. Use `Shift`+`Enter` to run individual code cells.
```

## Why use Python for image analysis?

- ✅ Easy to learn and use
- ✅ Free & open-source
- ✅ Wide range of libraries
- ✅ Implement custom functions & tools
- ✅ Automate image analysis via scripts
- ✅ Deep learning & data science
- ✅ Explore data in Juptyer notebooks

## Contents

We'll cover the following topics:

- <a href="#opening-our-first-image">Opening our first image</a>
- <a href="#how-are-digital-images-represented-in-python">How are digital images represented in Python?</a>
- <a href="#selecting-and-modifying-pixel-values">Selecting and modifying pixel values</a>
- <a href="#opening-images-from-files">Opening images from files</a>
- <a href="#applying-image-processing-filters">Applying image processing filters</a>
- <a href="#thresholding">Thresholding</a>
- <a href="#analyzing-individual-objects">Analyzing individual objects</a>

## Packages

We'll use the packages listed in the requirements file (<a href="./requirements.txt">requirements.txt</a>). To make sure you have installed these packages, you can run the Jupyter "magic" command below:

In [ ]:
%pip install -qr requirements.txt

## Imports

We start by importing the `data` module from <a href="https://scikit-image.org/">Scikit-image</a>, which provides example images, and Matplotlib's `pyplot` module which we'll use to display images.

In [ ]:
from skimage import data
import matplotlib.pyplot as plt

## Opening our first image

Let's load an example image provided by Scikit-image's `data` module. We'll display it using Matplotlib's `imshow` function.

In [ ]:
image = data.chelsea()

plt.imshow(image);

## How are digital images represented in Python?

Let's print the `type` of our image variable.

In [ ]:
type(image)

We've managed to read the image into a **Numpy array**. Numpy arrays sit at the foundation of the scientific Python ecosystem, and are also the most common way of representing images in Python.

Our image is really just an array of numbers:

In [ ]:
image

We can inspect the array's **attributes**, such as its `dtype` (data type), which tells us what kind of numbers are stored in the image array:

In [ ]:
image.dtype

Common types include **integers** such as 8-bit unsigned (`uint8`) used to store integer values between 0 and 255. There are also **floating-point** types (`float32`, `float64`) that store arrays of decimal numbers. Arrays encoded in a higher number of bits (32, 64) take up more space in memory, but allow to store larger numbers and to perform certain computations with more precision.

Next, we can examine the image's `shape` attribute, which gives us the size of the pixel grid:

In [ ]:
image.shape

In this case, we have loaded an image of 300 x 451 pixels, which also contains three color channels (RGB).

We can display the color channels separately. Below, we create a `Figure` object with subplots for the red, green and blue channels, respectively:

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(16, 8), ncols=3)
ax1.imshow(image[:, :, 0], cmap="gray")
ax2.imshow(image[:, :, 1], cmap="gray")
ax3.imshow(image[:, :, 2], cmap="gray")
plt.show()

Or convert the image to grayscale (the gray value is calculated as: `Y = 0.2125 R + 0.7154 G + 0.0721 B`):

In [ ]:
from skimage.color import rgb2gray

gray_image = rgb2gray(image)

plt.imshow(gray_image);

print("Image shape: ", gray_image.shape)
print("Data type: ", gray_image.dtype)
print("Min value: ", gray_image.min())
print("Max value: ", gray_image.max())

By default, Matplotlib uses the `viridis` **colormap** to display images. We can change the colormap by specifying a value for the `cmap` parameter of the `imshow` function (<a href="https://matplotlib.org/stable/users/explain/colors/colormaps.html">docs</a>).

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(16, 8), ncols=3)
ax1.imshow(gray_image, cmap="viridis")
ax2.imshow(gray_image, cmap="gray")
ax3.imshow(gray_image, cmap="inferno")
plt.show()

If we want to use the "gray" colormap by default instead of "viridis", we can set:

In [ ]:
import matplotlib

matplotlib.rc("image", cmap="gray")

Finally, we can add a color bar to display the range of pixel values, and set the minimum and maximum values of the colorbar with the `vmin` and `vmax` parameters.

In [ ]:
fig, ax = plt.subplots(dpi=200)
im = ax.imshow(gray_image, vmin=0, vmax=1)
ax.set_axis_off()  # Hide the axes
fig.colorbar(im)
plt.show()

```{note} Beyond Matplotlib
Matplotlib is suitable for many simple visualization cases. However, other visualization libraries exist in Python. For example:

- <a href="https://napari.org/">Napari</a>
- <a href="https://plotly.com/python/imshow/">Plotly</a>
- <a href="https://github.com/haesleinhuepf/stackview?tab=readme-ov-file">Stackview</a>
```

## Selecting and modifying pixel values

Since images are just Numpy arrays, we can apply any array operations to them. To select a subset of pixels in a region of interest in the image, we can use Numpy **indexing** along each axis, using the syntax `a[start:stop:step]`

In the example below, we "crop" the image along X and Y using this technique:

In [ ]:
crop = image[70:160, 120:230]

plt.imshow(crop);

Similarly, we can select and change pixel values in our image. For example:

In [ ]:
image[50:100, 50:100] = [255, 0, 0]

plt.imshow(image);

```{note} Image coordinates
The origin of the image coordinates is in the top-left corner of the image. To remember this, we can imagine the image as a table of values, where the first axis corresponds to rows and the second axis to columns of pixels.

![coordinates](./assets/image-coordinates.png)
```

## Opening images from files

To work with real scientific images in Python, the first step is usually to open them from files and load the image data into memory as Numpy arrays.

To do this, we need to use a library that can recognize the file format that we are dealing with. For common file formats (e.g. TIFF) we can use the `imread` function from Scikit-image.

In [ ]:
import skimage.io

# Image source: VIATICALPES (https://catima.unil.ch/viatimages/en)
image = skimage.io.imread("limat_fish.tif")

plt.imshow(image);

print("Image shape: ", image.shape)
print("Data type: ", image.dtype)
print("Min value: ", image.min())
print("Max value: ", image.max())

```{note} Image formats
To open certain specific formats (e.g. microscopy or astronomy data formats), we may need to install and use dedicated libraries.

Below are a few examples.

| Files             | Examples                        | Library |
|-------------------|---------------------------------| ------ |
| Common formats | `.tif`, `.png`, `.jpg` | `scikit-image`, `Pillow`, `tifffile`, `OpenCV` |
| Biological formats  |  `.czi`, `.lif`, `.ome.tiff` | `bioio` |
| Specific formats  | `.mrc`, `.hdf5`, `.fits` | `mrcfile`, `h5py`, `fitsio` |
```

Once again, we have loaded our image into a Numpy array. Notice that this is a grayscale image, with a size of 722 x 1037 pixels.

## Applying image processing filters

Image processing filters are operations applied to the whole image, for example to remove noise, sharpen details, detect edges, or rescale the image intensity values.

The scikit-image <a href="https://scikit-image.org/docs/0.25.x/api/skimage.filters.html">filters</a> module offers a wide range of image processing filters to choose from. For example, to apply a Gaussian filter:

In [ ]:
from skimage.filters import gaussian

blurred_img = gaussian(image, sigma=4.0)

plt.imshow(blurred_img);

The intensity of the blur is controlled by the value of `sigma`, the scale of the Gaussian kernel. We can try differnt values of `sigma` to understand its effect:

In [ ]:
sigmas = [0, 1, 4, 8]

n_sigmas = len(sigmas)

fig, axes = plt.subplots(figsize=(16, 4), ncols=n_sigmas)
for sigma, ax in zip(sigmas, axes):
    blurred_img = gaussian(image, sigma=sigma)
    ax.set_title(f"{sigma=}")
    ax.set_axis_off()
    ax.imshow(blurred_img)
plt.show()

````{admonition} Where to find documentation?

In the code, we can always access a function's documentation using the `help` function (for example: `help(gaussian)`).

It is also useful to search for a function via the Scikit-image <a href="https://scikit-image.org/docs/0.25.x/api/api.html">API reference</a>, which links to usage examples from the Scikit-image <a href="https://scikit-image.org/docs/stable/auto_examples">Gallery</a>.
````

## Thresholding

Thresholding ("binarizing") an image based on pixel values helps us separate the foreground from the background of the image based on intensity values. The result of applying a threshold is a **binary segmentation mask** that can be used to locate, count, or anlayze the size and shape of structures of interest.

To determine a suitable threshold to apply to our image, let's take a look at the histogram of pixel intensity values.

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), ncols=2)
ax1.imshow(image)
ax2.hist(image.flatten(), bins=50, log=True)
plt.show()

With Numpy, we can effectively create a **boolean mask** from the image array, where pixels satisfying a given condition are set to `True` and other pixels are set to `False`.

In the case of thresholding, this technique involves selecting all pixels greater (or lower) than a given value:

In [ ]:
binary_img = image < 200

binary_img

In [ ]:
plt.imshow(binary_img, interpolation="none");

````{note} Automatic thresholding

The example above works fine, however we selected our threshold rather arbitrarily. To improve reproducibility, we could use **automatic threshold algorithms** such as <a href="https://en.wikipedia.org/wiki/Otsu's_method">Otsu's method</a> to automatically estimate an appropriate threshold value.

With Scikit-image, you can try:

```
from skimage.filters import threshold_otsu

thresh = threshold_otsu(image)

binary_img = image > thresh
```
````

We can further process our binary mask by filling in the morphological "holes" (here, we use an implementation from the <a href="https://docs.scipy.org/doc/scipy/reference/ndimage.html">scipy.ndimage</a> module).

In [ ]:
from scipy.ndimage import binary_fill_holes

binary_img = binary_fill_holes(binary_img)

plt.imshow(binary_img);

Let's also remove "small objects", i.e. groups of foreground pixels smaller than a predefined `min_size` value. 

In [ ]:
from skimage.morphology import remove_small_objects

binary_img = remove_small_objects(binary_img, min_size=500)

plt.imshow(binary_img);

Our binary mask now seems acceptable. From it, we can calculate, for example, the area fraction of positive pixels:

In [ ]:
area_fract = binary_img.sum() / binary_img.size

print("Fraction of `True` pixels: ", area_fract)

## Analyzing individual objects

To separate individual objects in our binary image, we can apply an algorithm known as **connected components labeling**. This algorithms identifies disconnected groups of pixels and assigns a different *label* value to each of them.

In a "labeled" image array, values respresent object instances (the background is `0` by convention).

In [ ]:
from skimage.measure import label

label_img = label(binary_img)

plt.imshow(label_img, cmap="gnuplot");

Finally, Scikit-image provides functions for measuring object properties (shape and intensity features) in a labeled image (<a href="https://scikit-image.org/docs/0.25.x/api/skimage.measure.html#skimage.measure.regionprops">docs</a>).

We can represent the results in a tabular format, such as a <a href="https://pandas.pydata.org/">Pandas DataFrame</a>.

In [ ]:
import pandas as pd
from skimage.measure import regionprops_table

measurements = regionprops_table(label_img, intensity_image=image, properties=["label", "area", "centroid", "image_intensity"])

table = pd.DataFrame(measurements)

table.sort_values(by="area", ascending=False, inplace=True)

table

Working with a labeled array allows us to select only pixels belonging to a single object. For example:

In [ ]:
row = table.iloc[0]

plt.imshow(row.image_intensity)
plt.title(f"Biggest fish: {row.area} px.")
plt.show()

Another way of selecting values in an image array is via **filtering**.

Here, we create a binary mask with `True` values in the selected object. We then create a copy of the original image and use our mask to set the pixel values *outside* the mask to zero.

In [ ]:
selected_object_mask = label_img == row.label

big_fish_image = image.copy()
big_fish_image[~selected_object_mask] = 0

pos_x = row["centroid-0"]
pos_y = row["centroid-1"]

plt.imshow(big_fish_image)
plt.title(f"Centroid position: ({pos_x:.0f}, {pos_y:.0f})")
plt.scatter(pos_y, pos_x, s=50)
plt.show()

As we can see, there's not *always* a bigger fish!

## Conclusion

In this notebook, we've covered the basics of how to:

- Open images using `imread` and display them using `imshow`.
- Access array **attributes** (`shape`, `dtype`...) and **methods** (`min()`, `max()`...).
- Use **indexing** (`image[10:20, 10:30]`) and **filtering** (`image > 50`) to select and modify pixels from an image array.
- Find and use image processing functions and filters from Scikit-image.
- Generate and work with **binary masks** and **labeled images**.
- Measure and analyze individual **object properties**.

## Where to go next?

We've just scratched the surface of how Python can be used for image analysis. There is much more to learn!

Official documentation pages:

- <a href="https://scikit-image.org/docs/stable/">Scikit-image’s documentation</a>
- <a href="https://numpy.org/doc/2.3/">NumPy documentation</a>
- <a href="https://matplotlib.org/stable/index.html">Matplotlib documentation</a>

From the <a href="https://imaging.epfl.ch/">EPFL Center for Imaging</a>:

- <a href="https://epfl-center-for-imaging.github.io/python-setup/">Setting up Python for scientific image analysis</a> - Our short guide on how to to install, set up, and work with Python on your system.
- <a href="https://github.com/EPFL-Center-for-Imaging/awesome-scientific-image-analysis">Awesome Image Analysis</a> - Our curated list of online learning resources and image analysis tools.